# ERC-4626: vaults across all chains

This is a [research notebook](https://tradingstrategy.ai/glossary/jupyter-notebook) for listing and analysing ERC-4626 vaults.

- In this notebook, we examine different [ERC-4626 vaults](https://tradingstrategy.ai/glossary/erc-4626) across different [EVM blockchains](https://tradingstrategy.ai/glossary/evm-compatible)
    - Currently we do not scan non-ERC-4626 vaults like Enzyme Finance, or any protocol-native vaults like Hyperliquid HPL. This is not an inherit limitation, this is not just yet implemented.
- We assemble various data tables out of the vault data to show and compare the blockchain ecosystems
- The analysis focus on USD-stablecoin nonminatd vaults
    - Currently missing are e.g. WETH vaults and staking vaults for various small cap tokens
    - There is no ERC standard for vaults fees - for some protocols we have manualled added fee reading support  
- The list of chains is somewhat randomly selected and very easy to extend to contain any chain supported by [Envio's HyperSync](https://docs.envio.dev/docs/HyperSync/hypersync-supported-networks)
- Everything is open source: You can run this notebook and associated scripts yourself on your local computer, it will take around an hour

In this notebook, we use terms Net Asset Value (NAV) and [Total Value Locked (TVL)](https://tradingstrategy.ai/glossary/total-value-locked-tvl) interchangeably.

## Usage

- First you need the data. See :ref:`scan-erc-4626` to get the data locally.
- Then just run this notebook in Visual Studio Code,

## Setup

- Set up notebook renderinb parmaeters

In [13]:
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format

## Read scanned data

- Read the Pickle database our scanning script produced earlier 

In [14]:
import pickle
from pathlib import Path

import pandas as pd

from eth_defi.token import is_stablecoin_like

output_folder = Path("~/.tradingstrategy/vaults").expanduser()
vault_db = output_folder / "vault-db.pickle"
assert vault_db.exists(), "Run the vault scanner script first"

vault_db = pickle.load(open(vault_db, "rb"))

print(f"We have data for {len(vault_db)} vaults")

We have data for 10357 vaults


## Transform data

- Prepare the raw vault pickled data as Pandas DataFrame for data research

In [15]:
import datetime
from pprint import pformat
import pandas as pd
from eth_defi.erc_4626.hypersync_discovery import ERC4262VaultDetection
from eth_defi.chain import get_chain_name
from eth_defi.token import is_stablecoin_like

data = list(vault_db.values())
df = pd.DataFrame(data)

# print("Raw row example:")
# print(df.iloc[0])

# Build useful columns out of raw pickled Python data
# _detection_data contains entries as ERC4262VaultDetection class
entry: ERC4262VaultDetection
df["Chain"] = df["_detection_data"].apply(lambda entry: get_chain_name(entry.chain))
df["Protocol identified"] = df["_detection_data"].apply(lambda entry: entry.is_protocol_identifiable())
df["Stablecoin denominated"] = df["_denomination_token"].apply(lambda token_data: is_stablecoin_like(token_data["symbol"]) if pd.notna(token_data) else False)
df["ERC-7540"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7540())
df["ERC-7575"] = df["_detection_data"].apply(lambda entry: entry.is_erc_7575())
df["Fee detected"] = df.apply(lambda row: (row["Mgmt fee"] is not None) or (row["Perf fee"] is not None), axis=1)
# Event counts
df["Deposit count"] = df["_detection_data"].apply(lambda entry: entry.deposit_count)
df["Redeem count"] = df["_detection_data"].apply(lambda entry: entry.redeem_count)
df["Total events"] = df["Deposit count"] + df["Redeem count"]
df["Mgmt fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Perf fee"] = df["Mgmt fee"].fillna("<unknown>")
df["Age"] = datetime.datetime.utcnow() - df["First seen"]
df["NAV"] = df["NAV"].astype("float64")

df = df.sort_values(["Chain", "Address"])
df = df.set_index(["Chain", "Address"])

print("DataFrame MultiIndex is:", ", ".join(x for x in df.index.names))
print("DataFrame columns are:", ", ".join(x for x in df.columns))

source_df = df

display(df.sort_values("Total events", ascending=False).head())

DataFrame MultiIndex is: Chain, Address
DataFrame columns are: Symbol, Name, Denomination, NAV, Protocol, Mgmt fee, Perf fee, Shares, First seen, _detection_data, _denomination_token, _share_token, Protocol identified, Stablecoin denominated, ERC-7540, ERC-7575, Fee detected, Deposit count, Redeem count, Total events, Age


,,Symbol,Name,Denomination,NAV,Protocol,Mgmt fee,Perf fee,Shares,First seen,_detection_data,...,_share_token,Protocol identified,Stablecoin denominated,ERC-7540,ERC-7575,Fee detected,Deposit count,Redeem count,Total events,Age
Chain,Address,,,,,,,,,,,,,,,,,,,,,
Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,BBiBGT,Beraborrow iBGT,iBGT,"147,889.43",<unknown ERC-4626>,<unknown>,<unknown>,82754.082127361669804141,2025-03-07 12:05:19,"ERC4262VaultDetection(chain=80094, address='0x...",...,"{'name': 'Beraborrow iBGT', 'symbol': 'BBiBGT'...",False,False,False,False,False,556860,511476,1068336,135 days 22:42:09.276133
Gnosis,0xaf204776c7245bf4147c2612bf6e5972ee483701,sDAI,Savings xDAI,WXDAI,"113,397,116.45",<unknown ERC-4626>,<unknown>,<unknown>,95352751.939834307120594939,2023-09-28 12:13:30,"ERC4262VaultDetection(chain=100, address='0xaf...",...,"{'name': 'Savings xDAI', 'symbol': 'sDAI', 'to...",False,True,False,False,False,441833,398719,840552,661 days 22:33:58.276133
Soneium,0x232554b4b291a446b4829300bec133fbb07a8f2a,UB-WETH,UB-WETH,WETH,54.09,<unknown ERC-4626>,<unknown>,<unknown>,53.87605275747257428,2025-01-13 06:53:05,"ERC4262VaultDetection(chain=1868, address='0x2...",...,"{'name': 'UB-WETH', 'symbol': 'UB-WETH', 'tota...",False,False,False,False,False,589769,204989,794758,189 days 03:54:23.276133
Berachain,0x69f1e971257419b1e9c405a553f252c64a29a30a,oriBGT,Origami iBGT Auto-Compounder,iBGT,"4,227,149.42",<unknown ERC-4626>,<unknown>,<unknown>,2354308.792247967168348693,2025-03-17 01:22:21,"ERC4262VaultDetection(chain=80094, address='0x...",...,"{'name': 'Origami iBGT Auto-Compounder', 'symb...",False,False,False,False,False,465439,128823,594262,126 days 09:25:07.276133
Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,yvUSDC-A,USDC yVault-A,USDC,"229,745.40",Yearn v3,<unknown>,<unknown>,193222.27032,2023-10-26 16:45:48,"ERC4262VaultDetection(chain=137, address='0xa0...",...,"{'name': 'USDC yVault-A', 'symbol': 'yvUSDC-A'...",True,True,False,False,False,465224,25293,490517,633 days 18:01:40.276133


## Vaults per chain summary

- Get a summary of scanned chains at what vaults they have
- *Generic* status means that we do not have classification rules to determine the protocol on which a particular ERC-4626 vault belongs
- *Broken* status means that we could not correctly extract ERC-4626 information out of a smart contract

To detect the protocol of a vault, we need to maintain a [manual rule list here](https://github.com/tradingstrategy-ai/web3-ethereum-defi/blob/master/eth_defi/erc_4626/classification.py). Not all protocols are supported at the moment. as there are too many protocols to manually examine and identify them. Open source contributions welcome.




In [ ]:
nav_threshold = 100_000
broken_max_nav_threshold = 999_000_000_000

# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True
erc_7575 = df["ERC-7575"] == True
fee_detected = df["Fee detected"] == True

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]

# Create the summary DataFrame
summary_df = pd.DataFrame(
    {
        "Total vaults detected": df.groupby(level="Chain").size(),
        "Total vault TVL/NAV in stable vaults": stablecoin_only_df.groupby(level="Chain")["NAV"].sum(),
        "Protocol correctly identified": df[identified_filter].groupby(level="Chain").size().astype(int),
        "Stablecoin denominated": df[stablecoin_denominated].groupby(level="Chain").size().astype(int),
        f"Notable stablecoin NAV (min {nav_threshold:,} USD)": df[notable_nav].groupby(level="Chain").size().astype(int),
        f"ERC-7540": df[erc_7540].groupby(level="Chain").size().astype(int),
        f"ERC-7575": df[erc_7575].groupby(level="Chain").size().astype(int),
        f"Fee data supported": df[fee_detected].groupby(level="Chain").size().astype(int),
    }
).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the
# NA is not present in the final results
print("Vault counts per feature per chain")
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Chain,,,,,,,,
Arbitrum,2097,"238,816,450.49",383.00,730,66.00,32.00,40.00,236.00
Avalanche,368,"355,555,033.39",64.00,111,22.00,1.00,1.00,48.00
Base,1814,"856,601,267.51",779.00,850,77.00,48.00,27.00,148.00
Berachain,713,"41,695,598.19",54.00,30,11.00,0.00,0.00,5.00
Binance,397,"400,065,292.90",61.00,117,27.00,0.00,0.00,27.00
Blast,107,"1,447,671.88",2.00,14,2.00,0.00,0.00,32.00
Celo,10,"3,215,217.23",0.00,7,1.00,0.00,0.00,0.00
Ethereum,2621,"12,952,657,137.38",749.00,953,283.00,30.00,28.00,258.00
Gnosis,93,"115,743,124.97",13.00,39,5.00,0.00,0.00,0.00


## Vaults per protocol summary

- Break down by identified protocol
- Because of the brokeness of EVM, Solidity and smart contract development practices, protocol-rules are hand-maintained heurestics and there can false positives and negatives

In [17]:
df = source_df.copy()
df = df.reset_index()
# Built different masks
identified_filter = df["Protocol identified"] == True
stablecoin_denominated = df["Stablecoin denominated"] == True
notable_nav = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
notable_usage = df["Stablecoin denominated"] & (df["NAV"] >= nav_threshold)
erc_7540 = df["ERC-7540"] == True
erc_7575 = df["ERC-7575"] == True
fee_detected = df["Fee detected"] == True

stablecoin_only_df = df[df["Stablecoin denominated"] == True]
stablecoin_only_df = stablecoin_only_df[stablecoin_only_df["NAV"] < broken_max_nav_threshold]
# Create the summary DataFrame
summary_df = pd.DataFrame(
    {
        "Total vaults detected": df.groupby("Protocol").size(),
        "Total vault TVL/NAV in stable vaults": stablecoin_only_df.groupby("Protocol")["NAV"].sum(),
        "Protocol correctly identified": df[identified_filter].groupby("Protocol").size().astype(int),
        "Stablecoin denominated": df[stablecoin_denominated].groupby("Protocol").size().astype(int),
        f"Notable stablecoin NAV (min {nav_threshold} USD)": df[notable_nav].groupby("Protocol").size().astype(int),
        f"ERC-7540": df[erc_7540].groupby("Protocol").size().astype(int),
        f"ERC-7575": df[erc_7575].groupby("Protocol").size().astype(int),
        f"Fee data supported": df[fee_detected].groupby("Protocol").size().astype(int),
    }
).fillna(0)

# TODO: Having NA in calculations somewhere confuses Pandas and makes int columns floats even if the
# NA is not present in the final results
print("Vault counts per feature per chain")
summary_df = summary_df.sort_values("Total vault TVL/NAV in stable vaults", ascending=False)
display(summary_df)

Vault counts per feature per chain


,Total vaults detected,Total vault TVL/NAV in stable vaults,Protocol correctly identified,Stablecoin denominated,Notable stablecoin NAV (min 100000 USD),ERC-7540,ERC-7575,Fee data supported
Protocol,,,,,,,,
<unknown ERC-4626>,7403,"10,925,702,438.04",0.00,"2,665.00",297.00,0.00,12.00,0.00
Morpho,273,"2,045,923,959.87",273.00,165.00,82.00,0.00,0.00,273.00
Euler Vault Kit,470,"631,964,915.22",470.00,124.00,46.00,0.00,0.00,0.00
Fluid,64,"577,751,393.68",64.00,26.00,8.00,0.00,0.00,0.00
<unknown ERC-7540>,50,"442,709,177.55",0.00,27.00,5.00,50.00,46.00,0.00
Yearn v3,172,"245,783,137.41",172.00,86.00,25.00,0.00,0.00,0.00
Kiln Metavault,93,"169,180,404.50",93.00,77.00,34.00,0.00,0.00,0.00
Superform,149,"26,484,463.98",149.00,60.00,11.00,20.00,0.00,0.00
Peapods,176,"23,359,719.98",176.00,77.00,13.00,0.00,0.00,0.00


## First and latest vaults on each chain

- Show how much history we have for each chain


In [18]:
# Assuming your DataFrame is named 'df'
df = source_df
seen_df = df.groupby(level="Chain")["First seen"].agg(["min", "max"]).reset_index()

# Rename columns for clarity
seen_df.columns = ["Chain", "First vault deployed", "Last vault deployed"]

seen_df = seen_df.set_index("Chain")

display(seen_df)

,First vault deployed,Last vault deployed
Chain,,
Arbitrum,2022-03-28 09:03:11,2025-07-17 14:54:04
Avalanche,2022-04-11 00:05:24,2025-07-16 16:54:05
Base,2023-08-04 15:36:07,2025-07-19 12:29:15
Berachain,2025-01-26 00:06:03,2025-07-17 18:44:00
Binance,2022-05-27 17:25:18,2025-07-17 13:26:48
Blast,2024-02-29 20:15:37,2024-12-03 15:38:15
Celo,2022-08-08 21:14:10,2025-05-05 13:09:47
Ethereum,2019-06-11 06:17:19,2025-07-18 16:34:11
Gnosis,2022-05-01 18:47:35,2025-06-26 13:54:40


## Largest USD vaults

- Show the stablecoin-denominated vaults across different chains that have largest USD treasury 

In [25]:
largest_threshold = 30
largest_df = df.reset_index()
# Filter out crap
largest_df = largest_df[largest_df["Total events"] > 100]
largest_df = largest_df[largest_df["Stablecoin denominated"] == True]
largest_df = largest_df.sort_values(["NAV"], ascending=False)

largest_df = largest_df[["NAV", "Chain", "Address", "Name", "Denomination", "Total events", "Mgmt fee", "Perf fee"]]
largest_df = largest_df.set_index("Name")


display(largest_df.head(largest_threshold))

,NAV,Chain,Address,Denomination,Total events,Mgmt fee,Perf fee
Name,,,,,,,
Staked USDe,"3,528,322,039.32",Ethereum,0x9d39a5de30e57443bff2a8307a4256c8797a3497,USDe,78350,<unknown>,<unknown>
Savings USDS,"1,803,768,785.11",Ethereum,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,USDS,52925,<unknown>,<unknown>
Syrup USDC,"869,227,368.02",Ethereum,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,USDC,11606,<unknown>,<unknown>
Ethereal Pre-deposit Vault,"672,783,246.26",Ethereum,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,USDe,83975,<unknown>,<unknown>
Spark USDC Vault,"555,442,025.32",Base,0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a,USDC,33786,0.00,0.00
Savings Dai,"511,951,190.65",Ethereum,0x83f20f44975d03b1b09e64809b757c47f942beea,DAI,77653,<unknown>,<unknown>
Staked Falcon USD,"297,722,940.41",Ethereum,0xc8cf6d7991f15525488b2a83df53468d682ba4b0,USDf,11023,<unknown>,<unknown>
Spark USDC Vault,"291,669,161.12",Ethereum,0xbc65ad17c5c0a2a4d159fa5a503f4992c7b545fe,USDC,4475,<unknown>,<unknown>
Compounding Open Dollar,"289,207,352.81",Ethereum,0xad55aebc9b8c03fc43cd9f62260391c13c23e7c0,USDO,2320,<unknown>,<unknown>


## Largest USD vault per chain

- Get the largest vault of each chain

In [20]:
# Get the index of max NAV for each chain
largest_df = largest_df.reset_index().set_index(["Chain", "Name"])
max_nav_idx = largest_df.groupby("Chain")["NAV"].idxmax()
# Use these indices to get the full rows
result = largest_df.loc[max_nav_idx]

display(result)

NAV  \
Chain     Name                                                                
Arbitrum  ostiumLP                                            42,919,541.00   
Avalanche Staked avUSD                                        85,912,657.51   
Base      Spark USDC Vault                                   555,442,025.32   
          Spark USDC Vault                                    11,386,042.17   
Berachain Concrete-USDe-Vault                                 13,441,206.33   
Binance   Staked USDX                                        234,904,024.91   
Blast     Wasabi USDB Vault                                      961,876.68   
Celo      Staked EURA                                          3,215,211.40   
Ethereum  Staked USDe                                      3,528,322,039.32   
Gnosis    Savings xDAI                                       113,397,116.45   
Mode      USDC Ironclad Vault                                    337,647.27   
Optimism  Spark USDC Vault                                    16,441,224.14   
Polygon   Compound USDC                                       25,183,585.70   
Soneium   UB-USDC                                                193,880.55   
Sonic     Silo Finance Borrowable USDC Deposit, SiloId: 20    13,368,893.55   
ZKsync    hsUSDC                                                 597,225.98   

                                                                                               Address  \
Chain     Name                                                                                           
Arbitrum  ostiumLP                                          0x20d419a8e12c45f88fda7c5760bb6923cee27f98   
Avalanche Staked avUSD                                      0x06d47f3fb376649c3a9dafe069b3d6e35572219e   
Base      Spark USDC Vault                                  0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a   
          Spark USDC Vault                                  0x3128a0f7f0ea68e7b7c9b00afa7e41045828e858   
Berachain Concrete-USDe-Vault                               0x59e24f42cae1b82c8b2dc79ea898f2f8b4986dfc   
Binance   Staked USDX                                       0x7788a3538c5fc7f9c7c8a74eac4c898fc8d87d92   
Blast     Wasabi USDB Vault                                 0x4bed2a922654cacc2be974689619768fabf24855   
Celo      Staked EURA                                       0x004626a008b1acdc4c74ab51644093b155e59a23   
Ethereum  Staked USDe                                       0x9d39a5de30e57443bff2a8307a4256c8797a3497   
Gnosis    Savings xDAI                                      0xaf204776c7245bf4147c2612bf6e5972ee483701   
Mode      USDC Ironclad Vault                               0x882fd369341fc435ad5e54e91d1ebc23b1fc6d4c   
Optimism  Spark USDC Vault                                  0xcf9326e24ebffbef22ce1050007a43a3c0b6db55   
Polygon   Compound USDC                                     0x781fb7f6d845e3be129289833b04d43aa8558c42   
Soneium   UB-USDC                                           0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41   
Sonic     Silo Finance Borrowable USDC Deposit, SiloId: 20  0x322e1d5384aa4ed66aeca770b95686271de61dc3   
ZKsync    hsUSDC                                            0xaf08a9d918f16332f22cf8dc9abe9d9e14ddcbc2   

                                                           Denomination  \
Chain     Name                                                            
Arbitrum  ostiumLP                                                 USDC   
Avalanche Staked avUSD                                            avUSD   
Base      Spark USDC Vault                                         USDC   
          Spark USDC Vault                                         USDC   
Berachain Concrete-USDe-Vault                                      USDe   
Binance   Staked USDX                                              USDX   
Blast     Wasabi USDB Vault                                        USDB   
Celo      Staked EURA                                            

## Most active vaults across all chains

- Determine vault activity by number of deposit and redeem events
- Based on all-time event count, not recent event count 
- Events may be driven by bots, so this may not reflect the popularity of a vault amount users


In [21]:
largest_threshold = 20
largest_df = df.reset_index().sort_values(["Total events"], ascending=False)

largest_df = largest_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

largest_df = largest_df.set_index("Name")

display(largest_df.head(largest_threshold))

,Total events,Chain,Address,Denomination,NAV,Age,Deposit count,Redeem count
Name,,,,,,,,
Beraborrow iBGT,1068336,Berachain,0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"147,889.43",135 days 22:42:09.276133,556860,511476
Savings xDAI,840552,Gnosis,0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"113,397,116.45",661 days 22:33:58.276133,441833,398719
UB-WETH,794758,Soneium,0x232554b4b291a446b4829300bec133fbb07a8f2a,WETH,54.09,189 days 03:54:23.276133,589769,204989
Origami iBGT Auto-Compounder,594262,Berachain,0x69f1e971257419b1e9c405a553f252c64a29a30a,iBGT,"4,227,149.42",126 days 09:25:07.276133,465439,128823
USDC yVault-A,490517,Polygon,0xa013fbd4b711f9ded6fb09c1c0d358e2fbc2eaa0,USDC,"229,745.40",633 days 18:01:40.276133,465224,25293
USDT yVault-A,458738,Polygon,0xbb287e6017d3deb0e2e65061e8684eab21060123,USDT,"357,120.92",591 days 17:45:08.276133,407833,50905
BathToken v1,297381,Optimism,0xe0e112e8f33d3f437d1f895cbb1a456836125952,USDC,"58,548.84",1153 days 11:25:24.276133,150209,147172
Bridged USDC (Stargate)Vault,280214,Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,USDC.e,0.03,176 days 10:41:25.276133,123752,156462
UB-USDC,256764,Soneium,0xc675bb95d73ca7db2c09c3dc04daaa7944ccba41,USDC.e,"193,880.55",189 days 03:55:17.276133,180405,76359


## Most historically active vault per chain

- Vaults with most deposit and redeem events

In [22]:
most_active_df = df.reset_index()

most_active_df = most_active_df[["Total events", "Chain", "Address", "Name", "Denomination", "NAV", "Age", "Deposit count", "Redeem count"]]

# Force thousand separator
most_active_df["Total events"] = most_active_df["Total events"].astype("float64")

max_nav_idx = most_active_df.groupby("Chain")["Total events"].idxmax()
# Use these indices to get the full rows
result = most_active_df.loc[max_nav_idx]

result = result.set_index(["Chain", "Name"])

display(result)

,,Total events,Address,Denomination,NAV,Age,Deposit count,Redeem count
Chain,Name,,,,,,,
Arbitrum,Staked USDA,"124,094.00",0x0022228a2cc5e7ef0274a7baa600d44da5ab5776,USDA,"575,083.91",557 days 02:15:03.276133,121186,2908
Avalanche,Wrapped Aave Avalanche WAVAX,"181,356.00",0xd7da0de6ef4f51d6206bf2a35fcd2030f54c3f7b,WAVAX,"69,744.31",102 days 08:55:25.276133,99194,82162
Base,Moonwell Flagship USDC,"231,820.00",0xc1256ae5ff1cf2719d4937adb3bbccab2e00a2ca,USDC,"50,206,490.67",403 days 21:50:13.276133,127724,104096
Berachain,Beraborrow iBGT,"1,068,336.00",0xe59ab0c3788217e48399dae3cd11929789e4d3b2,iBGT,"147,889.43",135 days 22:42:09.276133,556860,511476
Binance,kUSDT,"43,544.00",0x1c3f35f7883fc4ea8c4bca1507144dc6087ad0fb,VUSD,"2,405,334.73",770 days 03:38:45.276133,25469,18075
Blast,Wasabi WETH Vault,"243,720.00",0x8e2b50413a53f50e2a059142a9be060294961e40,WETH,530.11,507 days 14:31:51.276133,161350,82370
Celo,Wrapped mcUSD,260.00,0x5f71fe197fd9aad95b7b5510a98882769622112a,mcUSD,"32,006.80",1039 days 15:13:31.276133,180,80
Ethereum,pufETH,"169,896.00",0xd9a442856c234a39a81a089c06451ebaa4306a72,WETH,"49,852.49",536 days 15:02:53.276133,152517,17379
Gnosis,Savings xDAI,"840,552.00",0xaf204776c7245bf4147c2612bf6e5972ee483701,WXDAI,"113,397,116.45",661 days 22:33:58.276133,441833,398719


## Oldest vaults

- Show oldest vaults

In [23]:
threshold = 1_000

oldest_df = df.reset_index()

oldest_df = oldest_df[["Chain", "Address", "Name", "Age", "Denomination", "NAV", "Total events"]]

# Force thousand separator
oldest_df["Total events"] = oldest_df["Total events"].astype("float64")

# Force event threshold to filter out some crap
oldest_df = oldest_df[oldest_df["Total events"] >= threshold]

max_nav_idx = oldest_df.groupby("Chain")["Age"].idxmax()
# Use these indices to get the full rows
result = oldest_df.loc[max_nav_idx]

result = result.set_index("Chain")

display(result)

,Address,Name,Age,Denomination,NAV,Total events
Chain,,,,,,
Arbitrum,0xf46ce0c13577232d5f29d9bd78a9cab278755346,Jones ETH,1150 days 19:32:22.276133,WETH,17.97,"2,383.00"
Avalanche,0x9dd17f32fc8355ae37425f475a10cc7bec8ca36a,,1194 days 17:50:06.276133,,0.00,"1,425.00"
Base,0xc7548d8d7560f6679e369d0556c44fe1eddea3e9,FARM_WETH,689 days 18:53:35.276133,WETH,0.73,"1,129.00"
Berachain,0x90bc07408f5b5eac4de38af76ea6069e1fcee363,Bridged USDC (Stargate)Vault,176 days 10:41:25.276133,USDC.e,0.03,"280,214.00"
Binance,0x0f8754b36a767c5579178bd8a04d2fcd9d530b70,ygNRCH,1119 days 13:54:49.276133,NRCH,"1,173,720.58","1,200.00"
Blast,0x8e2b50413a53f50e2a059142a9be060294961e40,Wasabi WETH Vault,507 days 14:31:51.276133,WETH,530.11,"243,720.00"
Ethereum,0x815c23eca83261b6ec689b60cc4a58b54bc24d8d,vTHOR,1182 days 23:21:51.276133,THOR,"74,645,708.84","20,623.00"
Gnosis,0x004626a008b1acdc4c74ab51644093b155e59a23,Staked EURA,688 days 21:05:33.276133,EURA,"491,171.30","3,698.00"
Mantle,0x8529019503c5bd707d8eb98c5c87bf5237f89135,Karak - mETH,453 days 16:35:22.276133,mETH,372.33,"13,388.00"


## All vaults

- List of all stablecoin vaults 
- We also use this list to show the vault which participate the stablecoin benchmark

In [24]:
all_df = df.reset_index()

min_nav = 10_000

stablecoin_denominated = all_df["Stablecoin denominated"] == True
all_df = all_df[stablecoin_denominated]
all_df = all_df[all_df["NAV"] < broken_max_nav_threshold]
all_df = all_df[all_df["NAV"] >= min_nav]
all_df = all_df[["NAV", "Chain", "Protocol", "Address", "Name", "Age", "Denomination", "Total events"]]
all_df = all_df.set_index("Name").sort_values("NAV", ascending=False)

print(f"Displaying {len(all_df)} stablecoin vaults w/min NAV of {min_nav} USD")
pd.options.display.max_rows = None
display(all_df)

Displaying 801 stablecoin vaults w/min NAV of 10000 USD


,NAV,Chain,Protocol,Address,Age,Denomination,Total events
Name,,,,,,,
Staked USDe,"3,528,322,039.32",Ethereum,<unknown ERC-4626>,0x9d39a5de30e57443bff2a8307a4256c8797a3497,608 days 02:41:17.276133,USDe,78350
Savings USDS,"1,803,768,785.11",Ethereum,<unknown ERC-4626>,0xa3931d71877c0e7a3148cb7eb4463524fec27fbd,306 days 21:10:41.276133,USDS,52925
Syrup USDC,"869,227,368.02",Ethereum,<unknown ERC-4626>,0x80ac24aa929eaf5013f6436cda2a7ba190f5cc0b,400 days 16:16:17.276133,USDC,11606
Ethereal Pre-deposit Vault,"672,783,246.26",Ethereum,<unknown ERC-4626>,0x90d2af7d622ca3141efa4d8f1f24d86e5974cc8f,157 days 19:33:17.276133,USDe,83975
Spark USDC Vault,"555,442,025.32",Base,Morpho,0x7bfa7c4f149e7415b73bdedfe609237e29cbf34a,186 days 14:16:13.276133,USDC,33786
Savings Dai,"511,951,190.65",Ethereum,<unknown ERC-4626>,0x83f20f44975d03b1b09e64809b757c47f942beea,845 days 00:22:41.276133,DAI,77653
Janus Henderson Anemoy Treasury Fund,"432,723,136.13",Ethereum,<unknown ERC-7540>,0x36036ffd9b1c6966ab23209e073c68eb9a992f50,104 days 13:35:41.276133,USDC,8
Staked Falcon USD,"297,722,940.41",Ethereum,<unknown ERC-4626>,0xc8cf6d7991f15525488b2a83df53468d682ba4b0,158 days 13:54:29.276133,USDf,11023
Spark USDC Vault,"291,669,161.12",Ethereum,<unknown ERC-4626>,0xbc65ad17c5c0a2a4d159fa5a503f4992c7b545fe,137 days 19:47:17.276133,USDC,4475
